# Base ResNet50 Model

No Regularization techniques applied. 

Batch Size: 256

In [1]:
import matplotlib.pyplot as plt
from matplotlib.patches import  Rectangle
import tensorflow as tf

import os
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, MaxPool2D,MaxPooling2D, Flatten,BatchNormalization, Dropout,ZeroPadding2D, AveragePooling2D, Add, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras import activations
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

from tensorflow.keras.datasets import cifar10
from keras.utils import to_categorical

import time
import pickle

Using TensorFlow backend.


In [2]:
#Define the Model
#Removing all regularizers

def res_identity(x, filters, num_batchnorm = 0, num_dropout=0, dropout_prob = 0):
  #renet block where dimension doesnot change.
  #The skip connection is just simple identity conncection
  #we will have 3 blocks and then input will be added

    x_skip = x # this will be used for addition with the residual block 
    f1, f2 = filters
    bn = num_batchnorm
    drp = num_dropout

    #first block 
    x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    if bn>0:
        x = BatchNormalization()(x)
        bn-=1
    x = Activation(activations.relu)(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1

    #second block # bottleneck (but size kept same with padding)
    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    if bn>0:
        x = BatchNormalization()(x)
        bn-=1
    x = Activation(activations.relu)(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1

    # third block activation used after adding the input
    x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    if bn>0:
        x = BatchNormalization()(x)
        bn-=1
    x = Activation(activations.relu)(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1

    # add the input 
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)


    return x

def res_conv(x, s, filters, num_batchnorm = 0, num_dropout=0, dropout_prob = 0):
    x_skip = x
    f1, f2 = filters
    bn = num_batchnorm
    drp = num_dropout

    # first block
    x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid')(x)
    # when s = 2 then it is like downsizing the feature map
    if bn>0:
        x = BatchNormalization()(x)
        bn-=1
    x = Activation(activations.relu)(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1

    # second block
    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    if bn>0:
        x = BatchNormalization()(x)
        bn-=1
    x = Activation(activations.relu)(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1

    #third block
    x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    if bn>0:
        x = BatchNormalization()(x)
        bn-=1

    # shortcut 
    x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)

    # add 
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)

    return x

def resnet50(num_batchnorm = 0, bn_pooling = False, dropout_prob=0, num_dropout_conv = 0, num_dropout_id = 0, num_dropout=0):

    input_im = Input(shape=(32, 32, 3)) # cifar 10 images size
    x = ZeroPadding2D(padding=(3, 3))(input_im)
    bn = num_batchnorm
    drp = num_dropout

    # 1st stage
    # here we perform maxpooling, see the figure above

    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
    if bn_pooling:
        x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1

    #2nd stage 
    # frm here on only conv block and identity block, no pooling

    x = res_conv(x, s=1, filters=(64, 256),num_batchnorm=bn, num_dropout = num_dropout_conv, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(64, 256),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(64, 256),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)

    # 3rd stage

    x = res_conv(x, s=2, filters=(128, 512),num_batchnorm=bn, num_dropout = num_dropout_conv, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(128, 512),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(128, 512),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(128, 512),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)

    # 4th stage

    x = res_conv(x, s=2, filters=(256, 1024),num_batchnorm=bn, num_dropout = num_dropout_conv, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(256, 1024),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(256, 1024),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(256, 1024),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(256, 1024),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(256, 1024),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)

    # 5th stage

    x = res_conv(x, s=2, filters=(512, 2048),num_batchnorm=bn, num_dropout = num_dropout_conv, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(512, 2048),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(512, 2048),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)

    # ends with average pooling and dense connection

    x = AveragePooling2D((2, 2), padding='same')(x)

    x = Flatten()(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1
    x = Dense(10, activation='softmax')(x) #multi-class

    # define the model 

    model = Model(inputs=input_im, outputs=x, name='Resnet50')

    return model

In [3]:
#to measure Time to Accuracy
#https://keras.io/guides/writing_your_own_callbacks/

class timeToAccuracy(keras.callbacks.Callback):
    def __init__(self, startTime, epoch_ckpt, model_name):
        super(timeToAccuracy, self).__init__()
        self.targetAcc = 0.87 #CHANGE TO 0.87 WHEN RUNNING MODEL
        self.foundTarget = False
        self.startTime = startTime
        self.epoch = 0
        self.epoch_ckpt = epoch_ckpt
        self.model_name = model_name
        self.prev_loss = None
    def on_epoch_end(self, epoch, logs=None):
        if self.epoch % self.epoch_ckpt == 0:
            print(self.epoch, epoch)
            name = self.model_name + '.h5'
            self.model.save_weights(name)
            print('end', logs)
            
        self.epoch += 1
#         if (self.prev_loss == None):
#             self.prev_loss = logs['loss']
#         else:
#             delta = np.abs(logs['loss'] - self.prev_loss)
        if not self.foundTarget:
            if logs['accuracy'] >= self.targetAcc:
                current = time.time()
                print("Time to reach {} accuracy: {} s".format(self.targetAcc, current-self.startTime))
                with open('{}_tta.pkl'.format(model_name), 'wb') as file:
                    pickle.dump(current-self.startTime, file)
                self.foundTarget = True

In [5]:
def fit_resnet(model, xtrain, ytrain, xtest, ytest, model_name, convergence=False):
  
    EPOCHS = 500 if convergence else 100     #Change to 200 when running the model
    BATCH_SIZE= 256
    VERBOSITY = 0   #Change to 0 when actually running model
    EPOCH_CKPT = 15 # save model every N epochs

    opt = keras.optimizers.Adam(learning_rate=1e-3)

    model.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=['accuracy'])
    #fit the model
    start = time.time()
    print('Fitting with BS ', BATCH_SIZE)
    es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20) if convergence else None
    model.fit(
        xtrain,
        ytrain,
        batch_size=BATCH_SIZE,
        verbose=VERBOSITY,
        epochs = EPOCHS,
        callbacks=[timeToAccuracy(startTime=start, epoch_ckpt=EPOCH_CKPT, model_name=model_name), es]
    )
    train_time = time.time() - start

    #evaluate
    score = model.evaluate(xtest,ytest)
    loss = score[0]
    acc = score[1]

    return train_time,  acc

In [4]:
#prepare data

(x_train, y_train), (x_test, y_test) = cifar10.load_data() 

# Data normalization
x_train = x_train/255
x_test = x_test/255
y_train = to_categorical(y_train) 
y_test = to_categorical(y_test)

In [6]:
dp = 0
prob = 0
bn = 0
print('Training BN{}, DPID{}, DPCONV{}, DPPROB{} till convergence'.format(bn, dp, dp, prob))
model_name = 'model_bn{}_dpid_{}_dpconv_{}_dpprob_{}_convergence'.format(bn, dp, dp, prob)
resnet_model = resnet50(num_batchnorm=bn, bn_pooling=True, num_dropout_conv=dp, num_dropout_id=dp, dropout_prob=prob)
tt, acc = fit_resnet(resnet_model,x_train, y_train, x_test, y_test, model_name, convergence=True)

print('Final accuracy {} reached in {}'.format(acc, tt))

Training BN0, DPID0, DPCONV0, DPPROB0 till convergence
Fitting with BS  256
0 0
end {'loss': 1.4863908290863037, 'accuracy': 0.47808000445365906}
Time to reach 0.87 accuracy: 82.88674354553223 s
15 15
end {'loss': 0.11034703999757767, 'accuracy': 0.9618399739265442}
30 30
end {'loss': 0.06647584587335587, 'accuracy': 0.9775000214576721}
45 45
end {'loss': 0.04945744574069977, 'accuracy': 0.9835000038146973}
60 60
end {'loss': 0.04485958814620972, 'accuracy': 0.9847000241279602}
75 75
end {'loss': 0.042905908077955246, 'accuracy': 0.9858800172805786}
90 90
end {'loss': 0.030663523823022842, 'accuracy': 0.9897199869155884}
105 105
end {'loss': 0.03622721880674362, 'accuracy': 0.9877200126647949}
120 120
end {'loss': 0.02734898030757904, 'accuracy': 0.990559995174408}
135 135
end {'loss': 0.03131677955389023, 'accuracy': 0.9892799854278564}
313/313 [==============================] - 3s 8ms/step - loss: 1.8440 - accuracy: 0.6980
Final accuracy 0.6980000138282776 reached in 1030.29130554199